In [1]:
import sys
sys.path.append('../')
from utils.process_data_online import *

In [23]:
from utils.features import *

In [2]:
def chunks_from_files(data, metadata):
    sess_data = pd.read_csv(data,header=None)
    sess_metadata = pd.read_csv(metadata, names=['start_time','category'])
    category_instances = {}
    for i, row in sess_metadata.iterrows():
        start_time = row['start_time']
        category = row['category'].strip()
        end_time = sess_metadata.iloc[i + 1]['start_time'] if i < (len(sess_metadata) - 1) else sess_data.iloc[-1][0]
        channels = sess_data.loc[(sess_data[0] >= start_time) & (sess_data[0] < end_time)]
        instance = np.array(channels)
        if category not in category_instances:
            category_instances[category] = []
        category_instances[category].append(instance)
    return category_instances

In [3]:
data_dir = '../data/emg_recordings/'
sess8 = data_dir + 'test_sess8_data.txt'
sess8_md = data_dir + 'test_sess8_metadata.txt'
chunk_dict = chunks_from_files(sess8, sess8_md)

In [4]:
chunk_dict.keys()

dict_keys(['REST', 'SNAP', 'CLENCH FIST'])

In [21]:
def compute_peaks(dict_key):
    peaks_list = []
    for chunk in chunk_dict[dict_key]:
        peaks = find_peaks(process_chunk(chunk))
        #print(peaks)
        peaks_list.append(peaks)
    overall_peaks = np.mean(np.array(peaks_list),axis=0)
    return overall_peaks

In [37]:
def compute_mav(dict_key):
    mav_list = []
    for chunk in chunk_dict[dict_key]:
        pro_chunk = process_chunk(chunk)
        mav_metric = np.mean([mean_absolute_value(pro_chunk[:,i]) for i in range(pro_chunk.shape[1])])
        mav_list.append(mav_metric)
    overall_mav = np.mean(mav_list)
    return overall_mav

In [38]:
print(compute_peaks('REST'))

[ 4.69700938 32.60445231 57.8268742  92.79905684]


In [39]:
print(compute_mav('REST'))

20.097930719680406


In [9]:
rest_peaks = np.mean(np.array(peaks_list),axis=0)
rest_peaks

array([ 4.69700938, 32.60445231, 57.8268742 , 92.79905684])

In [11]:
clench_list = []
for clench_chunk in chunk_dict['CLENCH FIST']:
    peaks = find_peaks(process_chunk(clench_chunk))
    print(peaks)
    clench_list.append(peaks)

[  4.91861397  35.10478385  50.92212474 128.23754623]
[  4.89433981  33.1682175   69.39005656 113.55751844]
[ 4.68646453 26.57184879 52.58866434 75.03340508]
[  4.56926007  34.90931478  61.03910682 107.47272661]
[  4.56206347  33.4606176   58.1486672  106.99895931]
[  4.68316739  36.88194163  54.01053865 106.61727544]
[ 4.65637048 29.75731461 53.12530962 74.69304205]
[ 4.71269782 31.16959654 67.98959028 84.38216012]
[ 4.70253784 26.19197185 59.60189478 78.38904895]
[ 4.72711898 30.0286388  51.01460445 81.61135603]
